# Azure AI Content Safety SDK Tutorial

This notebook demonstrates how to implement content safety checks for health and medical advice.

## Prerequisites
- Azure subscription with AI services access
- Python environment with required packages
- Basic understanding of Azure AI concepts

## What You'll Learn
- Initializing the content safety client
- Analyzing text for safety concerns
- Handling safety check results
- Best practices for health content

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.contentsafety import ContentSafetyClient
import os
import json

# Initialize client
try:
    client = ContentSafetyClient(
        endpoint=os.getenv("AZURE_CONTENTSAFETY_ENDPOINT"),
        credential=DefaultAzureCredential()
    )
    print("✓ Successfully initialized ContentSafetyClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## Content Safety Analysis

Let's analyze some health-related content:

In [ ]:
async def check_content_safety(text: str):
    """Analyze text for safety concerns."""
    try:
        result = await client.analyze_text(
            text=text,
            categories=["Hate", "SelfHarm", "Violence"]
        )
        
        # Check severity levels
        concerns = []
        for category in result.categories:
            if category.severity > 2:
                concerns.append(f"{category.category}: {category.severity}")
        
        return {
            "safe": len(concerns) == 0,
            "concerns": concerns,
            "result": result
        }
    except Exception as e:
        return f"Error checking content: {str(e)}"

# Test content safety
sample_content = [
    "Maintain a balanced diet and exercise regularly for good health.",
    "Consult your doctor before starting any new diet or exercise program.",
    "Some health conditions require special dietary considerations."
]

for content in sample_content:
    print(f"
Analyzing content: {content}")
    result = await check_content_safety(content)
    print(f"Safety check result: {result}")

## Health Content Guidelines

Implement specific checks for health-related content:

In [ ]:
async def validate_health_content(content: str):
    """Validate health-related content."""
    try:
        # Check content safety
        safety_result = await check_content_safety(content)
        
        if not safety_result["safe"]:
            return {
                "valid": False,
                "reason": f"Safety concerns: {safety_result['concerns']}"
            }
        
        # Additional health-specific checks
        required_disclaimers = [
            "consult",
            "healthcare",
            "professional",
            "medical"
        ]
        
        has_disclaimer = any(term in content.lower() for term in required_disclaimers)
        
        return {
            "valid": has_disclaimer,
            "reason": "Missing medical disclaimer" if not has_disclaimer else "Content approved"
        }
    except Exception as e:
        return f"Error validating content: {str(e)}"

# Test health content validation
health_advice = [
    "For weight management, consult your healthcare provider about appropriate diet and exercise.",
    "Try this new miracle diet for instant results!",
    "Discuss any dietary changes with your medical professional."
]

for advice in health_advice:
    print(f"
Validating advice: {advice}")
    result = await validate_health_content(advice)
    print(f"Validation result: {result}")

## Best Practices

1. **Content Analysis**
   - Check all relevant categories
   - Consider context
   - Implement appropriate thresholds
   - Monitor false positives

2. **Health Content**
   - Require medical disclaimers
   - Validate professional references
   - Check for misleading claims
   - Monitor user feedback

3. **Safety Measures**
   - Regular content reviews
   - Update safety criteria
   - Document decisions
   - Maintain audit trails